Loading the data

In [2]:
from sklearn.datasets import load_digits

digits = load_digits()

data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

Extract necessary data

In [3]:
import numpy as np

X = []
y = []
for i in range(target.shape[0]):
    if target[i]==3:
        X.append(data[i])
        y.append(1)
    elif target[i]==8:
        X.append(data[i])
        y.append(-1)
    else: continue

y = np.array(y)
X = np.array(X)
ones = np.ones(X.shape[0],dtype=float)
ones = ones[:, np.newaxis]
X = np.concatenate((X, ones),axis=1)
print("Shape of X is:", X.shape)
print("Shape of y is:", y.shape)

Shape of X is: (357, 65)
Shape of y is: (357,)


Logistic Regression and Cross Validation

In [4]:
import sklearn as sk
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv = 100, Cs = 100, random_state=0, 
                        solver ="liblinear",
                        fit_intercept = False,
                        refit = True,
                        multi_class="ovr").fit(X,y)

print("The best value for C is:", clf.C_)
print("The score is:", clf.score)
print("The regularization parameter is:", 1/clf.C_)
print(clf.Cs)
reg_param = 1/clf.C_

The best value for C is: [0.00413201]
The score is: <bound method LogisticRegressionCV.score of LogisticRegressionCV(Cs=100, class_weight=None, cv=100, dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=None, penalty='l2', random_state=0,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0)>
The regularization parameter is: [242.01282648]
100


1.2 Optimization Methods

In [172]:
def sigmoid(z):
    sigma =  1/(1+np.exp(-z))
    return sigma


def gradient(beta, X, y):
    update = np.zeros(beta.shape[0])
    for i in range(X.shape[0]):
        update = update + sigmoid(y[i]*np.dot(X[i,:],beta)) * y[i] * X[i,:]
    grad = (1/reg_param) * beta + (1/y.shape[0]) * update
    return grad

def gradient_achita(beta, X, y):
    N = y.shape[0]
    temp = sigmoid(-y*np.dot(X,beta))*y
    #Here is the correction of the grad
    result = beta - 1.0/N*(np.dot(X.T,temp))
    return result

def predict(beta,X):
    decision = np.dot(X,beta)
    y_computed = np.sign(decision)
    return y_computed

def zero_ones_loss(y_prediction, y_truth):
    zero_ones_vec = 1/2 * (y_prediction - y_truth)
    return np.dot(zero_ones_vec , zero_ones_vec)
    

Gradien descent

In [376]:
def gradient_descent(m,beta,tau,X,y):
    for i in range(m):
        beta = beta - tau * gradient(beta,X,y)
    return beta


Stochastic gradient

In [350]:
def stochastic_gradient(m,beta,X,y,tau0,gamma):
    tau = tau0
    for i in range(m):
        tau = tau/(1+np.power(gamma,i))
        X_random = X[np.random.randint(0,X.shape[0]),:].reshape(1,65)
        beta = beta - tau * gradient(beta, X_random, y)
    return beta

zero_ones_loss(predict( stochastic_gradient(150,np.zeros(65),X,y,0.1,0.01) , X),y)

183.0

SG minibatch

In [373]:
def SG_mini_batch(m,beta,X,y,tau0,gamma,B):
    tau = tau0
    for i in range(m):
        tau = tau/(1+np.power(gamma,i))
        rows_random = np.random.choice(np.arange(X.shape[0]),B,replace = True)
        X_random = X[rows_random,:]
        beta = beta - tau * gradient(beta,X_random,y)
    return beta

beta = SG_mini_batch(150,np.zeros(65),X,y,0.1,0.01,20)
print("The loss is:", zero_ones_loss(predict(beta,X),y))

The loss is: 179.0


SG momentum

In [380]:
def SG_momentum(m,beta,X,y,tau0,gamma,mu):
    g = np.zeros(65)
    tau = tau0
    for i in range(m):
        tau = tau/(1+np.power(gamma,i))
        X_random = X[np.random.randint(0,X.shape[0]),:].reshape(1,65)
        g = mu*g +(1-mu)*gradient(beta,X_random,y)
        beta = beta - tau*g
    return beta

beta = SG_momentum(150,np.zeros(65),X,y,0.1,0.01,0.9)
print("The loss is:", zero_ones_loss(predict(beta,X),y))

The loss is: 183.0


ADAM

In [383]:
def ADAM(m,beta,X,y,tau):
    g = 0
    q = 1
    mu1 = 0.9
    mu2 = 0.999
    epsilon = 1e-8
    for i in range(m):
        X_random = X[np.random.randint(0,X.shape[0]),:].reshape(1,65)
        grad = gradient(beta,X_random,y)     
        g = mu1*g + (1-mu1)*grad
        q = mu2*q + (1-mu2)*(grad*grad)
        g_tilt = g/(1-np.power(mu1,i+1))
        q_tilt = q/(1-np.power(mu2,i+1))  
        beta = beta - tau/(np.sqrt(q_tilt)+epsilon) * g_tilt
    return beta



beta = ADAM(150,np.zeros(65),X,y,0.1)
print("The loss is:", zero_ones_loss(predict(beta,X),y))

The loss is: 183.0


stochastic average gradient

In [437]:
def SAG(m,beta,X,y,tau0,gamma):
    g_stored = []
    tau = tau0
    for i in range(y.shape[0]):
        g_stored.append( y[i]* X[i] * sigmoid(-y[i] * np.dot( X[i] , beta)))
    g_stored = np.array(g_stored)
    g = np.sum(g_stored,axis = 0)/y.shape[0]
    for j in range(m):
        tau = tau/(1+np.power(gamma,j))
        i = np.random.randint(0,g_stored.shape[0])
        g_update = y[i] * np.dot( X[i] , sigmoid(-y[i]* np.dot(X[i] , beta)))
        g = g + 1/X.shape[0] *(g_update - g_stored[i])
        g_stored[i] = g_update
        beta = beta * (1- tau/reg_param) - tau*g 
    return beta

beta = SAG(150,np.zeros(65),X,y,0.1,0.01)
print("The loss is:", zero_ones_loss(predict(-beta,X),y))

The loss is: 20.0


dual coordinate ascent

In [307]:
def DCA(m):
    alpha = np.random.rand(y.shape[0])    
    beta = reg_param/y.shape[0] * np.dot( alpha * y, X)
    for j in range(m):
        i = np.random.randint(alpha.shape[0])
        f1 = y[i]*np.dot(X[i],beta) + np.log(alpha[i]/(1-alpha[i]))      
        f2 = reg_param/y.shape[0] * np.dot(X[i],X[i]) + 1/(alpha[i]*(1-alpha[i]))       
        alpha_new = alpha[i] - f1/f2
        beta = beta + (reg_param/y.shape[0]) *y[i]* X[i] * (alpha_new - alpha[i])
    return beta

beta = DCA(10)
print("The loss is:", zero_ones_loss(predict(beta,X),y))

The loss is: 70.0


In [343]:
def newton_raphson(m):
    for t in range(m):
        D = X.shape[1]
        N = X.shape[0]
        beta = np.zeros(D)
        z = np.dot( X , beta)
        y_tilt = y/sigmoid(np.dot(y,z))
        diag = np.array([reg_param/N*sigmoid(z[i])*sigmoid(-z[i]) for i in range(N)])
        W = np.diagflat(diag)
        beta = np.dot( np.dot( np.linalg.inv( np.identity(D) + np.dot ( np.dot(X.T , W) , X )) , np.dot(X.T,W) ), z + y_tilt)
    return beta

beta = newton_raphson(10)
print("The loss is:", zero_ones_loss(predict(beta,X),y))

The loss is: 0.0


In [340]:
L = [i for i in range(10)]
np.diagflat(L)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 4, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 5, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 6, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 7, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 8, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 9]])